# 🤖 Entraînement des Modèles ML - MarketPulse

## Objectif

Ce notebook entraîne les modèles ML pour la prédiction des prix (LSTM) et l'analyse de sentiment (FinBERT).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
import torch
import logging
import os

# Configuration
np.random.seed(42)
tf.random.set_seed(42)

# Configuration de l'affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Partie 1: Entraînement du modèle LSTM pour la prédiction des prix

In [ ]:
class LSTMModel:
    def __init__(self, sequence_length=60):
        self.sequence_length = sequence_length
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        
    def prepare_data(self, data, feature_columns=['Close']):
        """Préparer les données pour l'entraînement LSTM"""
        # Sélectionner les caractéristiques
        df = data[feature_columns].copy()
        
        # Normaliser les données
        scaled_data = self.scaler.fit_transform(df)
        
        # Créer des séquences
        X, y = [], []
        for i in range(self.sequence_length, len(scaled_data)):
            X.append(scaled_data[i-self.sequence_length:i])
            y.append(scaled_data[i, 0])  # Utiliser le prix de clôture comme cible
        
        X, y = np.array(X), np.array(y)
        
        # Diviser en ensembles d'entraînement et de test
        split_idx = int(0.8 * len(X))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        return X_train, X_test, y_train, y_test
    
    def build_model(self):
        """Construire le modèle LSTM"""
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(self.sequence_length, 1)),
            Dropout(0.2),
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model
    
    def train(self, X_train, y_train, epochs=50, batch_size=32):
        """Entraîner le modèle LSTM"""
        self.model = self.build_model()
        
        # Entraîner le modèle
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.1,
            verbose=1
        )
        
        return history
    
    def predict(self, X):
        """Faire des prédictions"""
        return self.model.predict(X)
    
    def evaluate(self, X_test, y_test):
        """Évaluer le modèle"""
        predictions = self.predict(X_test)
        
        # Inverser la transformation pour les prédictions et valeurs réelles
        pred_actual = self.scaler.inverse_transform(
            np.concatenate([predictions, np.zeros((predictions.shape[0], 4))], axis=1)
        )[:, 0]
        y_test_actual = self.scaler.inverse_transform(
            np.concatenate([y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 4))], axis=1)
        )[:, 0]
        
        mse = mean_squared_error(y_test_actual, pred_actual)
        mae = mean_absolute_error(y_test_actual, pred_actual)
        
        return mse, mae, pred_actual, y_test_actual

In [ ]:
# Charger les données ML préparées
symbol = "AAPL"
data = pd.read_csv(f'data/processed/{symbol}_ml_data.csv')
print(f"Données chargées: {data.shape}")
print(data.head())

In [ ]:
# Entraîner le modèle LSTM
lstm_model = LSTMModel(sequence_length=60)

# Préparer les données
X_train, X_test, y_train, y_test = lstm_model.prepare_data(data[['Close']])

# Reshape pour LSTM (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Entraîner le modèle
history = lstm_model.train(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Évaluer le modèle
mse, mae, pred_actual, y_test_actual = lstm_model.evaluate(X_test, y_test)
print(f"MSE: {mse}")
print(f"MAE: {mae}")

In [ ]:
# Visualiser les résultats
plt.figure(figsize=(15, 8))
plt.plot(y_test_actual, label='Valeurs réelles', color='blue')
plt.plot(pred_actual, label='Prédictions', color='red')
plt.title(f'Prédictions vs Réalité - {symbol}')
plt.xlabel('Temps')
plt.ylabel('Prix')
plt.legend()
plt.show()

In [ ]:
# Sauvegarder le modèle
model_dir = "models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
lstm_model.model.save(f"{model_dir}/lstm_model_{symbol}.h5")
print(f"Modèle LSTM sauvegardé: {model_dir}/lstm_model_{symbol}.h5")

## Partie 2: Entraînement du modèle FinBERT pour l'analyse de sentiment

In [ ]:
class SentimentModel:
    def __init__(self, model_name="ProsusAI/finbert"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.classifier = None
        
    def create_sample_data(self):
        """Créer des données d'exemple pour l'entraînement"""
        # Dans un scénario réel, vous chargeriez des données réelles
        # Pour cet exemple, nous créons des données synthétiques
        texts = [
            "The company reported strong quarterly earnings, exceeding analyst expectations.",
            "Market volatility increases as trade tensions escalate between major economies.",
            "New regulatory changes could impact the financial sector significantly.",
            "Stock prices surge following positive FDA approval for new drug.",
            "Economic indicators suggest a potential slowdown in the coming quarters.",
            "Company announces major acquisition that could transform its market position.",
            "Investors show caution amid uncertainty about future economic policies.",
            "Technology sector shows robust growth with new innovation breakthroughs.",
            "Oil prices drop due to oversupply concerns in the global market.",
            "Consumer spending increases, indicating strong economic confidence."
        ]
        
        # Labels: 0=négatif, 1=neutre, 2=positif
        labels = [2, 0, 1, 2, 0, 2, 1, 2, 0, 2]
        
        return pd.DataFrame({"text": texts, "label": labels})
    
    def tokenize_data(self, texts):
        """Tokeniser les textes"""
        return self.tokenizer(
            texts.tolist(),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
    
    def train(self, data, output_dir="./finbert_sentiment", epochs=3, batch_size=8):
        """Entraîner le modèle de sentiment"""
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import accuracy_score, precision_recall_fscore_support
        
        # Diviser les données
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            data["text"], data["label"], test_size=0.2, random_state=42
        )
        
        # Tokeniser les données
        train_encodings = self.tokenize_data(train_texts)
        val_encodings = self.tokenize_data(val_texts)
        
        # Créer une classe de dataset simple
        class FinancialNewsDataset(torch.utils.data.Dataset):
            def __init__(self, encodings, labels):
                self.encodings = encodings
                self.labels = labels

            def __getitem__(self, idx):
                item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
                item['labels'] = torch.tensor(self.labels.iloc[idx])
                return item

            def __len__(self):
                return len(self.labels)
        
        # Créer les datasets
        train_dataset = FinancialNewsDataset(train_encodings, train_labels.reset_index(drop=True))
        val_dataset = FinancialNewsDataset(val_encodings, val_labels.reset_index(drop=True))
        
        # Définir les arguments d'entraînement
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
        )
        
        # Créer le trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
        )
        
        # Entraîner le modèle
        trainer.train()
        
        # Sauvegarder le modèle
        model_dir = "models/finbert_sentiment"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        
        trainer.save_model(model_dir)
        self.tokenizer.save_pretrained(model_dir)
        
        # Créer le classifieur pour l'inférence
        self.classifier = pipeline(
            "sentiment-analysis",
            model="models/finbert_sentiment",
            tokenizer="models/finbert_sentiment"
        )
        
        return trainer

In [ ]:
# Entraîner le modèle de sentiment
sentiment_model = SentimentModel()
data = sentiment_model.create_sample_data()

print("Données d'entraînement:")
print(data)

# Entraîner le modèle (dans un environnement réel avec suffisamment de ressources)
# trainer = sentiment_model.train(data)

# Pour cet exemple, nous utiliserons le modèle pré-entraîné
classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# Tester le modèle
test_texts = [
    "The company's earnings exceeded expectations, showing strong growth.",
    "Market uncertainty continues to affect investor confidence.",
    "New product launch expected to drive revenue growth."
]

print("\nRésultats du modèle de sentiment:")
for text in test_texts:
    result = classifier(text)
    print(f"Texte: {text}")
    print(f"Sentiment: {result}\n")

## Conclusion

Ce notebook a entraîné deux modèles ML importants pour MarketPulse:

1. **Modèle LSTM** pour prédire les prix des actions
2. **Modèle FinBERT** pour l'analyse de sentiment

Les modèles sont maintenant prêts à être intégrés dans le pipeline de traitement de données.